# Zuehlke HW Assignment 2

## Done on a Mac, for a Mac


=====DAMLAS ASSIGNMENT # 2=====

Data Analytics and Machine Learning at Scale
Target, Minneapolis

ASSIGNMENT #2
Version 2016-06-26
Prepared by Dr. James G. Shanahan


SCHEDULE: This Homework is due by  Friday, July 8, 2016 at 11 AM (Central Time).


=== INSTRUCTIONS for SUBMISSIONS ===
Follow the instructions for submissions carefully.

Prepare a single Jupyter note, please include questions, and question numbers in the questions and in the responses.
Submit your homework notebook via the following form:

   + http://goo.gl/forms/er3OFr5eCMWDngB72


=== Other BACKGROUND INFORMATION
This Homework, you will need to use a map-reduce framework to solve all  problems involving data. You can use MrJob or raw Hadoop mapreduce streaming.

The following notebook serves as a useful Notebook to extend for this Homework (loaded with examples and boiler plate code)

   +  https://www.dropbox.com/s/pjd6maluq4ogt7m/HW02-Supporting-Material.ipynb?dl=0

Please refer to the following slides and reference material for more background


Pattern in MapReduce : 

    + Lin, Jimmy, & Dyer, Chris. (2010). Data-intensive text processing with MapReduce. 
      San Rafael, CA: Morgan & Claypool Publishers. 
      Chapter 3
      Book available here: https://lintool.github.io/MapReduceAlgorithms/MapReduce-book-final.pdf

   + Chapter 2 in this book: Hadoop with MRJob (calculater salare average etc.) 
          -  https://www.dropbox.com/s/jd3z2s216p9kc1z/hadoop-with-python-MRJOB.pdf?dl=0
          -  Source code: https://www.dropbox.com/sh/j8oettuxbgztk0p/AAAwq9PpEeByecDmaSNslnBPa?dl=0

   + http://mrjob.readthedocs.io/en/latest/

Counters in MRJob
   + http://nbviewer.jupyter.org/urls/dl.dropbox.com/s/5thl14n4pqvhzt5/Counter.ipynb 

KMeans Clustering (a form of Flat clustering)
   + Slides:  https://www.dropbox.com/s/6ef1mlrqr3xpnw0/Lecture07.1-Clustering.pdf?dl=0
   + http://nlp.stanford.edu/IR-book/pdf/16flat.pdf
   + http://nbviewer.jupyter.org/urls/dl.dropbox.com/s/dcw8evd9v0su3xu/K_Means_Unit_Test_Notebook.ipynb



=== SERVER with MRJob installed

http://ec2-52-201-222-181.compute-1.amazonaws.com:8000

 you will need a password to access.

Dont forget to save your notebooks REGULARLY to your local laptops (as the cluster could crash without warning)

# HW2.0
## How do you merge  two sorted  lists/arrays of records of the form [key, value]? Where is this  used in Hadoop MapReduce?

### _Assuming the lists are already sorted, compare the first element from each list and take the lesser element and add to a new sortedlist.  Then compare the larger of first elements with the next in the other, take the smaller and add to the sorted list. Repeat this process until both original lists have been completely parsed.  Here is some code to demonstrate further._

```
def mergeSortedLists(a,b):  #  Define class called mergeSortedLists with inputs a and b.  a and b are already sorted.
    SortedList = []  # Instantiate SortedList empty list.
    while a and b:   # Execute this loop until either a or b has been completely parsed.
        if a[0] < b[0] :  # If the first element of a is less than b...
            SortedList.append(a.pop(0)) # Take the first element of a and add it to SortedList.
        else:             # If the first element of a is not less than b...
            SortedList.append(b.pop(0)) # Take the first element of b and add it to SortedList.  Repeat this until both are empty.
    return SortedList + a + b  Return SortedList + any remaining elements of a and b (the very last one done).

```

### _The process is done in the shuffle phase, between the mapper and reducer, to further simplify the reducer's load by providing fewer, sorted lists to perform the reducer tasks on._

## What is  a combiner function in the context of Hadoop?  Give an example where it can be used and justify why it should be used in the context of this problem.

### _A combiner is used between mapper and reducer to reduce the data transfer between mapper and reducer as a sort of "pre reducer."  An example of where a combiner should be used (other than always) is with the Wordcount problem with a large dictionary.  Because there could be a large number of records coming from each mapper, the reducer could get overtaxed, which would lead to either a crash or a large amount of time to complete.  The combiner will act as an "in-mapper reducer" and consolidate the output from the mapper by combining similar keys.  This way, fewer records are passed to the reducer.  This speeds up the process and reduces the risk of system crash._

## What is the Hadoop shuffle?
### _The Hadoop shuffle is the process of moving key-value pairs from the a mapper to the reducer.  As data is being moved from mapper to reducer, it is being sorted to simplify the reducer's processing.  In the example of word count, a word appearing more than one time in the mapper's output to the reducer will be sorted together, so the reducer doesn't have to parse the entire input in order to aggregate._

# HW2.1
## Counters as a debugging aid (and for getting work done, but please use sparingly as they are heavy)

Consumer complaints dataset: Use Counters to do EDA (exploratory data analysis and to monitor progress)
Counters are lightweight objects in Hadoop that allow you to keep track of system progress in both the map and reduce stages of processing. By default, Hadoop defines a number of standard counters in "groups"; these show up in the jobtracker webapp, giving you information such as "Map input records", "Map output records", etc. 

While processing information/data using MapReduce job, it is a challenge to monitor the progress of parallel threads running across nodes of distributed clusters. Moreover, it is also complicated to distinguish between the data that has been processed and the data which is yet to be processed. The MapReduce Framework offers a provision of user-defined Counters, which can be effectively utilized to monitor the progress of data across nodes of distributed clusters.

Use the Consumer Complaints  Dataset provide here to complete this question:

     https://www.dropbox.com/s/vbalm3yva2rr86m/Consumer_Complaints.csv?dl=0

Use the following command to grab the file:

    curl -L https://www.dropbox.com/s/vbalm3yva2rr86m/Consumer_Complaints.csv?dl=0 -o Consumer_Complaints.csv

The consumer complaints dataset consists of diverse consumer complaints, which have been reported across the United States regarding various types of loans. The dataset consists of records of the form:

Complaint ID,Product,Sub-product,Issue,Sub-issue,State,ZIP code,Submitted via,Date received,Date sent to company,Company,Company response,Timely response?,Consumer disputed?

Here’s is the first few lines of the  of the Consumer Complaints  Dataset:

Complaint ID,Product,Sub-product,Issue,Sub-issue,State,ZIP code,Submitted via,Date received,Date sent to company,Company,Company response,Timely response?,Consumer disputed?
1114245,Debt collection,Medical,Disclosure verification of debt,Not given enough info to verify debt,FL,32219,Web,11/13/2014,11/13/2014,"Choice Recovery, Inc.",Closed with explanation,Yes,
1114488,Debt collection,Medical,Disclosure verification of debt,Right to dispute notice not received,TX,75006,Web,11/13/2014,11/13/2014,"Expert Global Solutions, Inc.",In progress,Yes,
1114255,Bank account or service,Checking account,Deposits and withdrawals,,NY,11102,Web,11/13/2014,11/13/2014,"FNIS (Fidelity National Information Services, Inc.)",In progress,Yes,
1115106,Debt collection,"Other (phone, health club, etc.)",Communication tactics,Frequent or repeated calls,GA,31721,Web,11/13/2014,11/13/2014,"Expert Global Solutions, Inc.",In progress,Yes,

User-defined Counters

Now, let’s use MapReduce Counters to identify the number of complaints pertaining to debt collection, mortgage and other categories (all other categories get lumped into this one) in the consumer complaints dataset. Basically produce the distribution of the Product column in this dataset using counters (limited to 3 counters here).

Hadoop offers Job Tracker, an UI tool to determine the status and statistics of all jobs. Using the job tracker UI, developers can view the Counters that have been created. Screenshot your  job tracker UI as your job completes and include it here. Make sure that your user defined counters are visible. 


### _Pull data from the website and load into Consumer__Complaints csv, loaded into memory._

In [4]:
!curl -L https://www.dropbox.com/s/vbalm3yva2rr86m/Consumer_Complaints.csv?dl=0 -o Consumer_Complaints.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:--  0:00:03 --:--:--     0
100 48.5M  100 48.5M    0     0   500k      0  0:01:39  0:01:39 --:--:--  661k


In [5]:
%%writefile ComplaintCountByTypeHW21.py

from mrjob.job import MRJob

class ComplaintCountByType(MRJob):
    
    def mapper(self, _, line):
        line = line.decode('utf8', 'ignore')
        
        complaint_id, product, _ = line.split(',', 2)
        del _

        if product.lower() != 'product':
            if product.lower() == 'debt collection':
                self.increment_counter('complaint_type', 'debt collection', 1)  
            elif product.lower() == 'mortgage':
                self.increment_counter('complaint_type', 'mortgage', 1)        
            else:
                self.increment_counter('complaint_type', 'other', 1) 
            

#The below combiners and reducers aren't used, but in order to keep programmatic consistency in alway using 
#mapper/reducer (always combiner!), I'm including them.

    def combiner(self, complainttype, complaintcount):
        yield complainttype, sum(complaintcount)

    def reducer(self, complainttype, complaintcount):
        yield complainttype, sum(complaintcount)
        
if __name__ == '__main__':
    ComplaintCountByType.run()

Overwriting ComplaintCountByTypeHW21.py


In [6]:
%reload_ext autoreload
%autoreload 2
from ComplaintCountByTypeHW21 import ComplaintCountByType
mr_job = ComplaintCountByType(args=['Consumer_Complaints.csv'])
with mr_job.make_runner() as runner: 
    runner.run()
    print runner.counters()

[{'complaint_type': {'debt collection': 44372, 'other': 142788, 'mortgage': 125752}}]


# HW2.2

## Analyze the performance of your Mappers, Combiners and Reducers using Counters

For this brief study the Input file will be one record (the next line only): 
foo foo quux labs foo bar quux

In [7]:
!echo foo foo quux labs foo bar quux > WordLineH22.txt

### HW2.2 Pt 1.1

Perform a word count analysis of this single record dataset using a Mapper and Reducer based WordCount (i.e., no combiners are used here) using user defined Counters to count up how many time the mapper and reducer are called. What is the value of your user defined Mapper Counter, and Reducer Counter after completing this word count job. The answer  should be 1 and 4 respectively. Please explain.

In [8]:
%%writefile MRWordCountFunCallHW2211.py

from mrjob.job import MRJob
import re
 
WORD_RE = re.compile(r"[\w']+")
 
class MRWordCountFunCall(MRJob):

    def mapper(self, _, line):
        self.increment_counter('FunctionCalls', 'Mapper', 1)
        for word in WORD_RE.findall(line):
            yield word.lower(), 1

    def reducer(self, word, counts):
        self.increment_counter('FunctionCalls', 'Reducer', 1)
        yield word, sum(counts)

if __name__ == '__main__':
    MRWordCountFunCall.run()

Writing MRWordCountFunCallHW2211.py


In [9]:
%reload_ext autoreload
%autoreload 2
from MRWordCountFunCallHW2211 import MRWordCountFunCall
mr_job = MRWordCountFunCall(args=['WordLineH22.txt'])
with mr_job.make_runner() as runner: 
    runner.run()
    print runner.counters()
    for line in runner.stream_output():
        print line.strip()

[{'FunctionCalls': {'Mapper': 1, 'Reducer': 4}}]
"bar"	1
"foo"	3
"labs"	1
"quux"	2


_If you look at the txt file WordLineHW22's contents, 1 line = 1 mapper.  There are four distinct words, so each word is sent individually to the reducer from the mapper, therefore 4 reducer calls._

### HW2.2 Pt 1.2
Perform a word count analysis of the Issue column of the Consumer Complaints  Dataset using a Mapper and Reducer based WordCount (i.e., no combiners used anywhere)  using user defined Counters to count up how many times the mapper and reducer are called. What is the value of your user defined Mapper Counter, and Reducer Counter after completing your word count job. 

In [10]:
%%writefile MRWordCountFunCallHW2212.py
from mrjob.job import MRJob
import re, string
 
class MRWordCountFunCall(MRJob):
    
    def mapper(self, _, line):
        
        self.increment_counter('FunctionCalls', 'Num_mapper_calls', 1)
        line = line.decode('utf8', 'ignore')
        
        complaint_id, complaint_type, sub_product, issue, _ = line.split(',', 4)

        regex = re.compile('[%s]' % re.escape(string.punctuation))
        issue = regex.sub(' ', issue.lower())
        issue = re.sub( '\s+', ' ', issue)

        issue = issue.split()
        
        try:
            int(complaint_id)
        except: 
            return
        
        del complaint_type, sub_product, _

        
        for word in issue:
            yield word, 1
        
    def reducer(self, word, counts):
        self.increment_counter('FunctionCalls', 'Num_reducer_calls', 1)
        yield word, sum(counts)
        
if __name__ == '__main__':
    MRWordCountFunCall.run()

Writing MRWordCountFunCallHW2212.py


In [11]:
%reload_ext autoreload
%autoreload 2
from MRWordCountFunCallHW2212 import MRWordCountFunCall
mr_job = MRWordCountFunCall(args=['Consumer_Complaints.csv'])
with mr_job.make_runner() as runner: 
    runner.run()
    counter = 1
    print runner.counters()
    for line in runner.stream_output():
        print line.strip()
        counter +=1
        if counter > 25:
            break

[{'FunctionCalls': {'Num_mapper_calls': 312913, 'Num_reducer_calls': 172}}]
"a"	3503
"account"	20681
"acct"	163
"action"	2505
"advance"	240
"advertising"	1193
"amount"	98
"amt"	71
"an"	2505
"and"	16448
"application"	8868
"applied"	139
"apply"	118
"apr"	3431
"arbitration"	168
"are"	3821
"atm"	2422
"attempts"	11848
"available"	274
"balance"	597
"bank"	202
"bankruptcy"	222
"being"	5663
"billing"	8158
"by"	5663


I only printed 25 records to keep the finished notebook smaller.  I could remove the following code from the above runner to print the entire list.

```
        counter +=1
        if counter > 25:
            break
```

Number of mapper calls: 312,913, Number of reducer calls: 172}}]

### HW2.2 Pt 1.3
Perform a word count analysis of the Issue column of the Consumer Complaints  Dataset using a Mapper, Reducer, and standalone combiner (i.e., not an in-memory combiner) based WordCount using user defined Counters to count up how many time the mapper, combiner, reducer are called.

In [12]:
%%writefile MRWordCountFunCallHW2213.py
from mrjob.job import MRJob
from mrjob.job import MRStep
import re, string
 
class MRWordCountFunCallCombiner(MRJob):
    
    def steps(self):
        return [
            MRStep(
                mapper=self.mapper,
            ),
            MRStep(   
                combiner=self.combiner
            ),
            MRStep(
                reducer=self.reducer  
            )
        ]
    
    def mapper(self, _, line):
        
        self.increment_counter('FunctionCalls', 'Num_mapper_calls', 1)
        line = line.decode('utf8', 'ignore')
        
        complaint_id, complaint_type, sub_product, issue, _ = line.split(',', 4)

        regex = re.compile('[%s]' % re.escape(string.punctuation))
        issue = regex.sub(' ', issue.lower())
        issue = re.sub( '\s+', ' ', issue)

        issue = issue.split()
        
        try:
            int(complaint_id)
        except: 
            return
        
        del complaint_type, sub_product, _

        for word in issue:
            yield word, 1
            
    def combiner(self, word, counts):
        self.increment_counter('FunctionCalls', 'Num_combiner_calls', 1)
        yield word, sum(counts)
        
    def reducer(self, word, counts):
        self.increment_counter('FunctionCalls', 'Num_reducer_calls', 1)
        yield word, sum(counts)
        
if __name__ == '__main__':
    MRWordCountFunCallCombiner.run()  

Writing MRWordCountFunCallHW2213.py


In [13]:
%reload_ext autoreload
%autoreload 2
from MRWordCountFunCallHW2213 import MRWordCountFunCallCombiner
mr_job = MRWordCountFunCallCombiner(args=['Consumer_Complaints.csv'])
with mr_job.make_runner() as runner: 
    runner.run()
    counter = 1
    print runner.counters()
    for line in runner.stream_output():
        print line.strip()
        counter +=1
        if counter > 25:
            break

[{'FunctionCalls': {'Num_mapper_calls': 312913}}, {'FunctionCalls': {'Num_combiner_calls': 483}}, {'FunctionCalls': {'Num_reducer_calls': 172}}]
"a"	3503
"account"	20681
"acct"	163
"action"	2505
"advance"	240
"advertising"	1193
"amount"	98
"amt"	71
"an"	2505
"and"	16448
"application"	8868
"applied"	139
"apply"	118
"apr"	3431
"arbitration"	168
"are"	3821
"atm"	2422
"attempts"	11848
"available"	274
"balance"	597
"bank"	202
"bankruptcy"	222
"being"	5663
"billing"	8158
"by"	5663


#### What is the value of your user defined Mapper Counter, combiner counter, and Reducer Counter after completing your word count job. 

Number of mapper calls:  312,913.  Number of reducer calls:  172.  Number of combiner calls:  172

I only printed 25 records to keep the finished notebook smaller.  I could remove the following code from the above runner to print the entire list.

```
        counter +=1
        if counter > 25:
            break
```

### HW2.2.1
Using a single reducer perform a sort of the words in decreasing order of word counts. Present the top 50 terms and their frequency. If there are ties please sort the tokens in alphanumeric/string order. 

HINT: You will need a second MRStep for the sort part. Step 1 will be the usual word count, while step 2 will be a sort step. Please use the Hadoop/MRJob framework to perform the sort. Please do NOT use any of the built-in sorts  from  python.

In [14]:
%%writefile MRWordCountFunCallMostFreqHW221.py
from mrjob.job import MRJob
from mrjob.job import MRStep
import re, string
 
class MRWordCountFunCallSorterMostFreq(MRJob):

    def steps(self):
        return [
            MRStep(
                mapper=self.mapper
               ,reducer=self.reducer
            )
             ,MRStep(
                 mapper=self.mapper_sort
                ,reducer=self.reducer_sort
            )
        ]

#MRStep 1
    def mapper(self, _, line):
        
        self.increment_counter('FunctionCalls', 'Num_mapper_calls',1)
        line = line.decode('utf8', 'ignore')
        
        complaint_id, complaint_type, sub_product, issue, _ = line.split(',', 4)

        regex = re.compile('[%s]' % re.escape(string.punctuation))
        issue = regex.sub(' ', issue.lower())
        issue = re.sub( '\s+', ' ', issue)

        issue = issue.split()
        
        try:
            int(complaint_id)
        except: 
            return
        
        del complaint_type, sub_product, _

        for word in issue:
            yield (word, 1)
            
    def reducer(self, word, counts):
        yield word, sum(counts)
        
#MRStep2
    def mapper_sort(self, word, count):
        yield (count-1e15, word), None
        
    def reducer_sort(self, countwords, _):
        count, word = countwords
        yield word, int(abs(count + 1e15))
        
if __name__ == '__main__':
    MRWordCountFunCallSorterMostFreq.run()  

Writing MRWordCountFunCallMostFreqHW221.py


In [15]:
from MRWordCountFunCallMostFreqHW221 import MRWordCountFunCallSorterMostFreq
mr_job = MRWordCountFunCallSorterMostFreq(args=['Consumer_Complaints.csv', '--jobconf="mapred.reduce.tasks=1"'])
with mr_job.make_runner() as runner: 
    runner.run()
    counter = 1
    for line in runner.stream_output():
        print counter, line.strip()
        counter += 1
        if counter > 50:
            break

1 "loan"	119630
2 "modification"	70487
3 "credit"	55251
4 "servicing"	36767
5 "report"	34903
6 "incorrect"	29133
7 "information"	29069
8 "on"	29069
9 "or"	22533
10 "account"	20681
11 "debt"	19309
12 "and"	16448
13 "opening"	16205
14 "club"	12545
15 "health"	12545
16 "not"	12353
17 "attempts"	11848
18 "collect"	11848
19 "cont"	11848
20 "d"	11848
21 "owed"	11848
22 "of"	10885
23 "my"	10731
24 "deposits"	10555
25 "withdrawals"	10555
26 "problems"	9484
27 "application"	8868
28 "to"	8401
29 "unable"	8178
30 "billing"	8158
31 "other"	7886
32 "disputes"	6938
33 "communication"	6920
34 "tactics"	6920
35 "reporting"	6559
36 "lease"	6337
37 "the"	6248
38 "being"	5663
39 "by"	5663
40 "caused"	5663
41 "funds"	5663
42 "low"	5663
43 "process"	5505
44 "disclosure"	5214
45 "verification"	5214
46 "managing"	5006
47 "company"	4858
48 "investigation"	4858
49 "s"	4858
50 "identity"	4729


Present bottom 10 tokens (least frequent items). 

In [16]:
%%writefile MRWordCountFunCallLeastFreqHW221.py
from mrjob.job import MRJob
from mrjob.job import MRStep
import re, string
 
class MRWordCountFunCallSorterLeastFreq(MRJob):
    
    def steps(self):
        return [
            MRStep(
                mapper=self.mapper
                ,reducer=self.reducer
            )
             ,MRStep(
                 mapper=self.mapper_sort
                ,reducer=self.reducer_sort
            )
        ]

#MRStep 1
    def mapper(self, _, line):
        
        self.increment_counter('FunctionCalls', 'Num_mapper_calls',1)
        line = line.decode('utf8', 'ignore')
        
        complaint_id, complaint_type, sub_product, issue, _ = line.split(',', 4)

        regex = re.compile('[%s]' % re.escape(string.punctuation))
        issue = regex.sub(' ', issue.lower())
        issue = re.sub( '\s+', ' ', issue)

        issue = issue.split()
        
        try:
            int(complaint_id)
        except: 
            return
        
        del complaint_type, sub_product, _
        
        for word in issue:
            if len(word)>1:
                yield (word, 1)
            
    def reducer(self, word, counts):
        yield word, sum(counts)
        
# #MRStep2
    def mapper_sort(self, word, counts):
        new_key = '%010d'%int(counts) + word 
        yield new_key, (word, counts)
        
    def reducer_sort(self, new_key, values):
        yield new_key,0
        
if __name__ == '__main__':
    MRWordCountFunCallSorterLeastFreq.run()  

Writing MRWordCountFunCallLeastFreqHW221.py


In [17]:
# #Ten Least Frequent
            
%reload_ext autoreload
%autoreload 2

from MRWordCountFunCallLeastFreqHW221 import MRWordCountFunCallSorterLeastFreq
mr_job = MRWordCountFunCallSorterLeastFreq(args=['Consumer_Complaints.csv', '--jobconf="mapred.reduce.tasks=1"'])
with mr_job.make_runner() as runner: 
    runner.run()
    counter = 1
    for line in runner.stream_output():
        keystring,dummy = line.split('\t')
        frequency = int(keystring[1:11])
        word = keystring[11:-1]
        print counter, (word, frequency)
        counter += 1
        if counter > 10:
            break

1 ('disclosures', 64)
2 ('missing', 64)
3 ('amt', 71)
4 ('day', 71)
5 ('checks', 75)
6 ('convenience', 75)
7 ('credited', 92)
8 ('payment', 92)
9 ('amount', 98)
10 ('apply', 118)


### HW2.2.2:   
Repeat HW2.2.1 using 3 reducers. Use the same code as in HW2.2.1  with just one modification 
to the command line: just add --jobconf mapred.reduce.tasks=3 as see presented here: 

    python HW2.2WordCount.py --jobconf mapred.reduce.tasks=3 oneLinerTextFile.txt

Top 50 with 3 reducers

In [18]:
from MRWordCountFunCallMostFreqHW221 import MRWordCountFunCallSorterMostFreq
mr_job = MRWordCountFunCallSorterMostFreq(args=['Consumer_Complaints.csv', '--jobconf="mapred.reduce.tasks=3"'])
with mr_job.make_runner() as runner: 
    runner.run()
    counter = 1
    for line in runner.stream_output():
        print counter, line.strip()
        counter += 1
        if counter > 50:
            break

1 "loan"	119630
2 "modification"	70487
3 "credit"	55251
4 "servicing"	36767
5 "report"	34903
6 "incorrect"	29133
7 "information"	29069
8 "on"	29069
9 "or"	22533
10 "account"	20681
11 "debt"	19309
12 "and"	16448
13 "opening"	16205
14 "club"	12545
15 "health"	12545
16 "not"	12353
17 "attempts"	11848
18 "collect"	11848
19 "cont"	11848
20 "d"	11848
21 "owed"	11848
22 "of"	10885
23 "my"	10731
24 "deposits"	10555
25 "withdrawals"	10555
26 "problems"	9484
27 "application"	8868
28 "to"	8401
29 "unable"	8178
30 "billing"	8158
31 "other"	7886
32 "disputes"	6938
33 "communication"	6920
34 "tactics"	6920
35 "reporting"	6559
36 "lease"	6337
37 "the"	6248
38 "being"	5663
39 "by"	5663
40 "caused"	5663
41 "funds"	5663
42 "low"	5663
43 "process"	5505
44 "disclosure"	5214
45 "verification"	5214
46 "managing"	5006
47 "company"	4858
48 "investigation"	4858
49 "s"	4858
50 "identity"	4729


Bottom 10 with 3 reducers

In [19]:
# #Ten Least Frequent
            
%reload_ext autoreload
%autoreload 2

from MRWordCountFunCallLeastFreqHW221 import MRWordCountFunCallSorterLeastFreq
mr_job = MRWordCountFunCallSorterLeastFreq(args=['Consumer_Complaints.csv', '--jobconf="mapred.reduce.tasks=3"'])
with mr_job.make_runner() as runner: 
    runner.run()
    counter = 1
    for line in runner.stream_output():
        keystring,dummy = line.split('\t')
        frequency = int(keystring[1:11])
        word = keystring[11:-1]
        print counter, (word, frequency)
        counter += 1
        if counter > 10:
            break

1 ('disclosures', 64)
2 ('missing', 64)
3 ('amt', 71)
4 ('day', 71)
5 ('checks', 75)
6 ('convenience', 75)
7 ('credited', 92)
8 ('payment', 92)
9 ('amount', 98)
10 ('apply', 118)


#### Describe what you see. Is this correct?

I see no difference in the output.  Running my code works the same with 1 or 3 reducers, which is to be expected.

# HW 2.2.3: 
## Optional; we will cover this in class

I've already solved the "total sort" issue in my code above :)

# HW2.3: Shopping Cart Analysis
Product Recommendations: The action or practice of selling additional products or services 
to existing customers is called cross-selling. Giving product recommendation is 
one of the examples of cross-selling that are frequently used by online retailers. 
One simple method to give product recommendations is to recommend products that are frequently
browsed together by the customers.
  
For this homework use the online browsing behavior dataset located at: 

       https://www.dropbox.com/s/zlfyiwa70poqg74/ProductPurchaseData.txt?dl=0

Each line in this dataset represents a browsing session of a customer. 
On each line, each string of 8 characters represents the id of an item browsed during that session. 
The items are separated by spaces.

Here are the first few lines of the ProductPurchaseData 
FRO11987 ELE17451 ELE89019 SNA90258 GRO99222 
GRO99222 GRO12298 FRO12685 ELE91550 SNA11465 ELE26917 ELE52966 FRO90334 SNA30755 ELE17451 FRO84225 SNA80192 
ELE17451 GRO73461 DAI22896 SNA99873 FRO86643 
ELE17451 ELE37798 FRO86643 GRO56989 ELE23393 SNA11465 
ELE17451 SNA69641 FRO86643 FRO78087 SNA11465 GRO39357 ELE28573 ELE11375 DAI54444 

Pull the data from the website

In [20]:
!curl -L https://www.dropbox.com/s/zlfyiwa70poqg74/ProductPurchaseData.txt?dl=0 -o ProductPurchaseData.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:--  0:00:03 --:--:--     0
100 3377k  100 3377k    0     0   370k      0  0:00:09  0:00:09 --:--:--  790k


### Do some exploratory data analysis of this dataset guided by the following questions:

#### Without using MapReduce... How many unique items are available from this supplier?  Assuming the full catalog has been bought within the scope of the ProductPurchaseData.txt file.

In [21]:
purchasedata = open('ProductPurchaseData.txt',"r+")

products = []

for session in purchasedata:
    prodID = session.split()
    for product in prodID:
        if product in products:
            continue
        else:
            products.append(product)
            
print "This supplier has " + str("{:,}".format(len(products))) + " unique products."

This supplier has 12,592 unique products.


# HW2.3 Part 1
## Using a single reducer, report your findings.  Use Hadoop MapReduce on the questions below.

###### What is the number of unique products?

In [22]:
%%writefile UniqueProductsHW231.py

from mrjob.job import MRJob
from mrjob.step import MRStep
import re, string

class MRJobUniqueProducts(MRJob):

    def steps(self):
        return [
            MRStep(
                mapper=self.mapper,
                reducer=self.reducer  
            )
        ]
    
    def mapper(self, _, line):
        
        regex = re.compile('[%s]' % re.escape(string.punctuation))
        session = regex.sub(' ', line.lower())
        session = re.sub( '\s+', ' ', session)
        session = session.split()
        
        for prod in session:
            yield (len(prod), prod)
    
    def combiner(self, key, values):
        yield key, sum(values)
            
    def reducer(self, key, values):
        yield "Number of Unique Products", len(set(values))

if __name__ == '__main__':
    MRJobUniqueProducts.run()

Writing UniqueProductsHW231.py


In [23]:
%reload_ext autoreload
%autoreload 2

from UniqueProductsHW231 import MRJobUniqueProducts

mr_job = MRJobUniqueProducts(args=['ProductPurchaseData.txt', '--jobconf="mapred.reduce.tasks=1"']) 
with mr_job.make_runner() as runner: 
    runner.run()
    for line in runner.stream_output():
        print line.strip()

"Number of Unique Products"	12592


This matches the number from above, so I know this is correct.

###### What is the largest basket?

In [24]:
%%writefile LargestBasketHW231.py

from mrjob.job import MRJob
from mrjob.step import MRStep
import re, string

class LargestBasket(MRJob):
    
    def steps(self):
        return [
            MRStep(
                mapper=self.mapper
            ),
            MRStep(
                mapper=self.mapper_sort,
                reducer=self.reducer_sort
            )
        ]
    
    def mapper(self, _, line):

        regex = re.compile('[%s]' % re.escape(string.punctuation))
        session = regex.sub(' ', line.lower())
        session = re.sub( '\s+', ' ', session)
        session = session.split()
        basket_size = len(session)

        yield ("Dummy", basket_size)
        
    def mapper_sort(self, basket, basket_size):
        yield ((basket_size-1e15, basket), None)
    
    def reducer_sort(self, sizebasket, _):
        size, basket = sizebasket
        yield "Largest Basket Size", int(abs(size + 1e15))
        
if __name__ == '__main__':
    LargestBasket.run()

Writing LargestBasketHW231.py


In [25]:
%reload_ext autoreload
%autoreload 2

from LargestBasketHW231 import LargestBasket
mr_job = LargestBasket(args=['ProductPurchaseData.txt', '--jobconf="mapred.reduce.tasks=1"']) 
with mr_job.make_runner() as runner: 
    runner.run()
    count = 0
    for line in runner.stream_output():
        print line.strip()
        count += 1
        if count > 0:
            break

"Largest Basket Size"	37


###### What are the top 50 most frequently purchased items and their frequency?  (Break ties by sorting products alphabetically.

In [26]:
%%writefile Top50ProductsHW231.py

from mrjob.job import MRJob
from mrjob.step import MRStep
import re, string

class Top50Products(MRJob):
    
    def steps(self):
        return [
            MRStep(
                mapper=self.mapper,
                combiner=self.combiner,
                reducer=self.reducer
            ),
            MRStep(
                mapper=self.mapper_sort,
                reducer=self.reducer_sort
            )
        ]
    
    def mapper(self, _, line):
        
        regex = re.compile('[%s]' % re.escape(string.punctuation))
        session = regex.sub(' ', line.lower())
        session = re.sub( '\s+', ' ', session)
        session = session.split()
        basket_size = len(session)
        
        for item in session:
            yield item, 1
        
    def combiner(self, item, counts):
        yield item, sum(counts)
        
    def reducer(self, item, counts):
        yield item, sum(counts)
        
#MRStep2
    def mapper_sort(self, word, count):
        yield (count-1e15, word), None
        
    def reducer_sort(self, countwords, _):
        count, word = countwords
        yield word.upper(), int(abs(count + 1e15))
                        
if __name__ == '__main__':
    Top50Products.run()

Writing Top50ProductsHW231.py


In [27]:
%reload_ext autoreload
%autoreload 2

from Top50ProductsHW231 import Top50Products
mr_job = Top50Products(args=['ProductPurchaseData.txt', '--jobconf="mapred.reduce.tasks=1"'])
with mr_job.make_runner() as runner: 
    runner.run()
    count = 0
    for line in runner.stream_output():
        print line.strip()
        count += 1
        if count > 49:
            break

"DAI62779"	6667
"FRO40251"	3881
"ELE17451"	3875
"GRO73461"	3602
"SNA80324"	3044
"ELE32164"	2851
"DAI75645"	2736
"SNA45677"	2455
"FRO31317"	2330
"DAI85309"	2293
"ELE26917"	2292
"FRO80039"	2233
"GRO21487"	2115
"SNA99873"	2083
"GRO59710"	2004
"GRO71621"	1920
"FRO85978"	1918
"GRO30386"	1840
"ELE74009"	1816
"GRO56726"	1784
"DAI63921"	1773
"GRO46854"	1756
"ELE66600"	1713
"DAI83733"	1712
"FRO32293"	1702
"ELE66810"	1697
"SNA55762"	1646
"DAI22177"	1627
"FRO78087"	1531
"ELE99737"	1516
"ELE34057"	1489
"GRO94758"	1489
"FRO35904"	1436
"FRO53271"	1420
"SNA93860"	1407
"SNA90094"	1390
"GRO38814"	1352
"ELE56788"	1345
"GRO61133"	1321
"DAI88807"	1316
"ELE74482"	1316
"ELE59935"	1311
"SNA96271"	1295
"DAI43223"	1290
"ELE91337"	1289
"GRO15017"	1275
"DAI31081"	1261
"GRO81087"	1220
"DAI22896"	1219
"GRO85051"	1214


# HW2.3.1 OPTIONAL 
Using 2 reducers:  Report your findings such as number of unique products; largest basket; report the top 50 most frequently purchased items, and their frequency (break ties by sorting the products alphabetical order) etc. using Hadoop Map-Reduce.

###### What is the number of unique products?

In [28]:
%reload_ext autoreload
%autoreload 2

from UniqueProductsHW231 import MRJobUniqueProducts

mr_job = MRJobUniqueProducts(args=['ProductPurchaseData.txt', '--jobconf="mapred.reduce.tasks=2"']) 
with mr_job.make_runner() as runner: 
    runner.run()
    for line in runner.stream_output():
        print line.strip()

"Number of Unique Products"	12592


###### What is the largest basket?

In [29]:
%reload_ext autoreload
%autoreload 2

from LargestBasketHW231 import LargestBasket
mr_job = LargestBasket(args=['ProductPurchaseData.txt', '--jobconf="mapred.reduce.tasks=2"']) 
with mr_job.make_runner() as runner: 
    runner.run()
    count = 0
    for line in runner.stream_output():
        print line.strip()
        count += 1
        if count > 0:
            break

"Largest Basket Size"	37


###### What are the top 50 most frequently purchased items and their frequency?  (Break ties by sorting products alphabetically.

In [30]:
%reload_ext autoreload
%autoreload 2

from Top50ProductsHW231 import Top50Products
mr_job = Top50Products(args=['ProductPurchaseData.txt', '--jobconf="mapred.reduce.tasks=2"'])
with mr_job.make_runner() as runner: 
    runner.run()
    counter = 1
    for line in runner.stream_output():
        print counter, line.strip()
        counter += 1
        if counter > 50:
            break

1 "DAI62779"	6667
2 "FRO40251"	3881
3 "ELE17451"	3875
4 "GRO73461"	3602
5 "SNA80324"	3044
6 "ELE32164"	2851
7 "DAI75645"	2736
8 "SNA45677"	2455
9 "FRO31317"	2330
10 "DAI85309"	2293
11 "ELE26917"	2292
12 "FRO80039"	2233
13 "GRO21487"	2115
14 "SNA99873"	2083
15 "GRO59710"	2004
16 "GRO71621"	1920
17 "FRO85978"	1918
18 "GRO30386"	1840
19 "ELE74009"	1816
20 "GRO56726"	1784
21 "DAI63921"	1773
22 "GRO46854"	1756
23 "ELE66600"	1713
24 "DAI83733"	1712
25 "FRO32293"	1702
26 "ELE66810"	1697
27 "SNA55762"	1646
28 "DAI22177"	1627
29 "FRO78087"	1531
30 "ELE99737"	1516
31 "ELE34057"	1489
32 "GRO94758"	1489
33 "FRO35904"	1436
34 "FRO53271"	1420
35 "SNA93860"	1407
36 "SNA90094"	1390
37 "GRO38814"	1352
38 "ELE56788"	1345
39 "GRO61133"	1321
40 "DAI88807"	1316
41 "ELE74482"	1316
42 "ELE59935"	1311
43 "SNA96271"	1295
44 "DAI43223"	1290
45 "ELE91337"	1289
46 "GRO15017"	1275
47 "DAI31081"	1261
48 "GRO81087"	1220
49 "DAI22896"	1219
50 "GRO85051"	1214


# HW2.4
## (Computationally prohibitive but then again Hadoop can handle this) Pairs

From a data mining perspective (and the aPriori algorihtm in particular), Support and Confidence are defined as follows:

       SUPPORT
       In data mining, the support value of X (where X is a collection of cooccurring items sometimes referred to as 
       an item-set. E.g., a basket or subset of a basket) with respect to T  (a transaction database where each 
       row is a transaction such as a basket of items that have been purchased)  is defined as the proportion 
       of transactions in the  database which contains  the item-set X. (a relative frequency of sorts) 

       CONFIDENCE 
       The confidence value of a rule, X ==>  Y (where X is a collection of cooccurring items and Y is generally 
       a single item. E.g., If Diapers and Beer then Cigars were also purchased), with respect to a set of transactions T, is the 
       proportion of the transactions that contains X which also contains Y. (Think of it as  tgePr(Y|X) )

       The pairs/stripes algorithm returns cooccurrence information that can be used directly to  calculate the confidence and support. 
       Note that confidence for pair X ==>  Y will  differ from the relative frequency that results from stripes when X occurs by itself in transactions.

# HW2.4.1
Suppose we want to recommend new products to the customer based on the products they
have already browsed on the online website. Write a map-reduce program 
to find products which are frequently browsed together. Fix the support count (cooccurence count) to s = 100 
(i.e. product pairs need to occur together at least 100 times to be considered frequent) 
and find pairs of items (sometimes referred to itemsets of size 2 in association rule mining) that have a support count of 100 or more.

In [31]:
%%writefile PairsHW241.py

from mrjob.job import MRJob
from mrjob.step import MRStep
import re, string

class Pairs(MRJob):
    
    def mapper(self, _, line):
        self.increment_counter('FunctionCalls', 'Mapper', 1)
        
        regex = re.compile('[%s]' % re.escape(string.punctuation))
        session = regex.sub(' ', line.lower())
        session = re.sub( '\s+', ' ', session)
        session = list(set(session.split()))
        session.sort()
        
        for i, item1 in enumerate(session):
            for item2 in session[i + 1:]:
                yield (item1, item2), 1
            
    def combiner(self, pair, counts):
        self.increment_counter('FunctionCalls', 'Combiner', 1)
        yield pair, sum(counts)
        
    def reducer(self, pair, counts):
        self.increment_counter('FunctionCalls', 'Reducer', 1)
                               
        count = sum(counts)
        if count > 99:
            yield pair, count

if __name__ == '__main__':
    Pairs.run()

Writing PairsHW241.py


In [32]:
%reload_ext autoreload
%autoreload 2

from PairsHW241 import Pairs
mr_job = Pairs(args=['ProductPurchaseData.txt']) 
with mr_job.make_runner() as runner: 
    runner.run()
    counter = 1
    for line in runner.stream_output():
        print line.strip()
        counter +=1
        if counter > 25:
            break

["dai16732", "fro78087"]	106
["dai18527", "sna44451"]	102
["dai22177", "dai31081"]	127
["dai22177", "dai62779"]	382
["dai22177", "dai63921"]	136
["dai22177", "dai75645"]	123
["dai22177", "dai83733"]	126
["dai22177", "dai85309"]	172
["dai22177", "ele17451"]	203
["dai22177", "ele26917"]	134
["dai22177", "ele32164"]	155
["dai22177", "ele34057"]	107
["dai22177", "ele56788"]	134
["dai22177", "ele66600"]	101
["dai22177", "ele66810"]	105
["dai22177", "ele74009"]	108
["dai22177", "ele91337"]	150
["dai22177", "fro31317"]	160
["dai22177", "fro32293"]	128
["dai22177", "fro40251"]	181
["dai22177", "fro66272"]	130
["dai22177", "fro78087"]	107
["dai22177", "fro80039"]	152
["dai22177", "fro85978"]	156
["dai22177", "gro21487"]	122


I only printed 25 records to keep the finished notebook smaller.  I could remove the following code from the above runner to print the entire list.

```
        counter +=1
        if counter > 25:
            break
```

# HW2.4.2
List the top 50 product pairs with corresponding support count (aka frequency), and relative frequency or support (number of records where they coccur, the number of records where they coccur/the number of baskets in the dataset)  in decreasing order of support  for frequent (100>count) itemsets of size 2. 

Use the Pairs pattern  to  extract these frequent itemsets of size 2. Free free to use combiners if they bring value. Instrument your code with counters for count the number of times your mapper, combiner and reducers are called.  

Please output records of the following form for the top 50 pairs (itemsets of size 2): 

      item1, item2, support count, support  (OPTIONAL Feel free to add in confidence level also)



Fix the ordering of the pairs lexicographically (left to right), 
and break ties in support (between pairs, if any exist) 
by taking the first ones in lexicographically increasing order. 

In [33]:
%%writefile PairsTop50HW242.py

from mrjob.job import MRJob
from mrjob.step import MRStep
import re, string

pairdata = {}

class Top50Pairs(MRJob):
    
    def steps(self):
        return [
            MRStep(
                mapper=self.mapper
                ,combiner=self.combiner
                ,reducer=self.reducer
            )
            ,MRStep(
                mapper=self.mapper_sort
                ,reducer=self.reducer_sort
            )
        ]
    
    def mapper(self, _, line):
        regex = re.compile('[%s]' % re.escape(string.punctuation))
        session = regex.sub(' ', line.lower())
        session = re.sub( '\s+', ' ', session)
        session = list(set(session.split()))
        session.sort()
        
        for i, item1 in enumerate(session):
            for item2 in session[i + 1:]:
                yield (item1, item2), 1
        yield ("", "total"), 1

    def combiner(self, pair, counts):
        yield pair, sum(counts)
        
    def reducer(self, pair, counts):
        count = sum(counts)
        
        if len(pair[0]) < 1:
            pairdata[pair[1]] = count
            return
        if count > 99:
            yield pair, count

    def mapper_sort(self, pair, count):

        yield (1e15 - count, pair), None      
        
    def reducer_sort(self, countpair, _):
        total = float(pairdata["total"])
        adjcount, pair = countpair
        frequency = int(abs(adjcount - 1e15))
        rel_freq = frequency / total
        frequencies = (frequency, rel_freq)
        yield pair, frequencies

if __name__ == '__main__':
    Top50Pairs.run()

Writing PairsTop50HW242.py


In [34]:
from PairsTop50HW242 import Top50Pairs
mr_job = Top50Pairs(args=['ProductPurchaseData.txt']) 
with mr_job.make_runner() as runner: 
    runner.run()
    counter = 1      
    for line in runner.stream_output():
        pair, frequencies =  mr_job.parse_output_line(line)
        print counter, pair[0].upper(), pair[1].upper(), frequencies[0], frequencies[1]
        counter += 1
        if counter > 50:
            break

1 DAI62779 ELE17451 1592 0.0511880646925
2 FRO40251 SNA80324 1412 0.0454004694383
3 DAI75645 FRO40251 1254 0.0403202469374
4 FRO40251 GRO85051 1213 0.0390019613517
5 DAI62779 GRO73461 1139 0.0366226166361
6 DAI75645 SNA80324 1130 0.0363332368734
7 DAI62779 FRO40251 1070 0.0344040384554
8 DAI62779 SNA80324 923 0.0296775023311
9 DAI62779 DAI85309 918 0.0295167357963
10 ELE32164 GRO59710 911 0.0292916626475
11 DAI62779 DAI75645 882 0.0283592167454
12 FRO40251 GRO73461 882 0.0283592167454
13 DAI62779 ELE92920 877 0.0281984502106
14 FRO40251 FRO92469 835 0.026848011318
15 DAI62779 ELE32164 832 0.0267515513971
16 DAI75645 GRO73461 712 0.0228931545609
17 DAI43223 ELE32164 711 0.022861001254
18 DAI62779 GRO30386 709 0.02279669464
19 ELE17451 FRO40251 697 0.0224108549564
20 DAI85309 ELE99737 659 0.0211890292917
21 DAI62779 ELE26917 650 0.020899649529
22 GRO21487 GRO73461 631 0.0202887366966
23 DAI62779 SNA45677 604 0.0194205974084
24 ELE17451 SNA80324 597 0.0191955242597
25 DAI62779 GRO71621 59

# HW2.4.3
Report  the compute time for the Pairs job. Describe the computational setup used (E.g., single computer; dual core; linux, number of mappers, number of reducers)
Instrument your mapper, combiner, and reducer to count how many times each is called using Counters and report these counts.

In [35]:
%%writefile PairsHW241_runnertimer.py
from PairsHW241 import Pairs
mr_job = Pairs(args=['ProductPurchaseData.txt', '--jobconf="mapred.map.tasks=2"', '--jobconf="mapred.reduce.tasks=2"']) 
with mr_job.make_runner() as runner: 
    runner.run()
    print runner.counters()

Writing PairsHW241_runnertimer.py


In [36]:
%time !python PairsHW241_runnertimer.py

[{'FunctionCalls': {'Mapper': 31101, 'Reducer': 877095, 'Combiner': 1026707}}]
CPU times: user 402 ms, sys: 127 ms, total: 529 ms
Wall time: 1min 8s


### Machine used was a quad core Mac, running on OS X.  Mapper/Reducer count set to 2.  Number of Mapper, Reducer, Combiner, MapperSort and ReducerSort calls in the results above.

# HW2.5
## Stripes

### HW2.5.1
Repeat 2.4.1 using the stripes design pattern for finding cooccuring pairs.

In [37]:
%%writefile StripesHW251.py

from mrjob.job import MRJob
from mrjob.step import MRStep
import re, string
from collections import Counter

stripedata = {}

class Stripes(MRJob):
    
    def steps(self):
        return [
            MRStep(
                mapper=self.mapper,
                combiner=self.combiner,
                reducer=self.reducer
            )
        ]
    
    def mapper(self, _, line):
        self.increment_counter('FunctionCalls', 'Mapper', 1)
        
        regex = re.compile('[%s]' % re.escape(string.punctuation))
        session = regex.sub(' ', line.lower())
        session = re.sub( '\s+', ' ', session)
        session = list(set(session.split()))
        session.sort()
        
        for i, item1 in enumerate(session):
            counts = Counter()
            for item2 in session[i + 1:]:
                counts[item2] += 1

            yield item1, counts
            
        yield "total", {'total': 1}

    def combiner(self, item, counters):
        self.increment_counter('FunctionCalls', 'Combiner', 1)
        counts = Counter()
        for c in counters: 
            for item2, count in c.items():
                counts[item2] += count
                
        yield item, counts
        
    def reducer(self, item, counters):
        self.increment_counter('FunctionCalls', 'Reducer', 1)
        
        counts = Counter()
        for c in counters: 
            for item2, count in c.items():
                counts[item2] += count
        
        if item == "total":
            stripedata[item] = counts[item]
            return
        
        for item2, count in counts.items():
            if count > 99:
                yield (item, item2), count

if __name__ == '__main__':
    Stripes.run()

Writing StripesHW251.py


In [38]:
%reload_ext autoreload
%autoreload 2

from StripesHW251 import Stripes
mr_job = Stripes(args=['ProductPurchaseData.txt']) 
with mr_job.make_runner() as runner: 
    runner.run()
    counter = 1
    for line in runner.stream_output():
        print line.strip()
        counter +=1
        if counter > 25:
            break

["dai16732", "fro78087"]	106
["dai18527", "sna44451"]	102
["dai22177", "gro21487"]	122
["dai22177", "sna55762"]	102
["dai22177", "fro80039"]	152
["dai22177", "dai85309"]	172
["dai22177", "ele91337"]	150
["dai22177", "dai62779"]	382
["dai22177", "ele17451"]	203
["dai22177", "dai75645"]	123
["dai22177", "gro73461"]	248
["dai22177", "ele32164"]	155
["dai22177", "sna80324"]	140
["dai22177", "gro71621"]	132
["dai22177", "dai63921"]	136
["dai22177", "fro78087"]	107
["dai22177", "gro30386"]	106
["dai22177", "fro31317"]	160
["dai22177", "dai31081"]	127
["dai22177", "ele74009"]	108
["dai22177", "gro46854"]	160
["dai22177", "fro85978"]	156
["dai22177", "gro59710"]	120
["dai22177", "fro40251"]	181
["dai22177", "fro66272"]	130


I only printed 25 records to keep the finished notebook smaller.  I could remove the following code from the above runner to print the entire list.

```
        counter +=1
        if counter > 25:
            break
```

# HW2.5.2
Stripes top 50.

Please output records of the following form for the top 50 pairs (itemsets of size 2): 

      item1, item2, support count, support  (OPTIONAL Feel free to add in confidence level also)

In [39]:
%%writefile StripesTop50HW252.py

from mrjob.job import MRJob
from mrjob.step import MRStep
import re, string
from collections import Counter

stripedata = {}

class StripesTop50(MRJob):
    
    def steps(self):
        return [
            MRStep(
                mapper=self.mapper,
                combiner=self.combiner,
                reducer=self.reducer
            ),
            MRStep(
                mapper=self.mapper_sort,
                reducer=self.reducer_sort
            )
        ]
    
    def mapper(self, _, line):
        self.increment_counter('FunctionCalls', 'Mapper', 1)
        
        regex = re.compile('[%s]' % re.escape(string.punctuation))
        session = regex.sub(' ', line.lower())
        session = re.sub( '\s+', ' ', session)
        session = list(set(session.split()))
        session.sort()
        
        for i, item1 in enumerate(session):
            counts = Counter()
            for item2 in session[i + 1:]:
                counts[item2] += 1

            yield item1, counts
            
        yield "total", {'total': 1}

    def combiner(self, item, counters):
        self.increment_counter('FunctionCalls', 'Combiner', 1)
        
        counts = Counter()
        for c in counters: 
            for item2, count in c.items():
                counts[item2] += count
        yield item, counts
        
    def reducer(self, item, counters):
        self.increment_counter('FunctionCalls', 'Reducer', 1)
        
        counts = Counter()
        for c in counters: 
            for item2, count in c.items():
                counts[item2] += count
        
        if item == "total":
            stripedata[item] = counts[item]
            return
        
        for item2, count in counts.items():
            if count > 99:
                yield (item, item2), count
                
    def mapper_sort(self, pair, count):
        self.increment_counter('FunctionCalls', 'MapperSort', 1)
        
        yield (1e15 - count, pair), None      
        
    def reducer_sort(self, countpair, _):
        self.increment_counter('FunctionCalls', 'Reducer', 1)

        total = float(stripedata["total"])
        adjcount, pair = countpair
        frequency = int(abs(adjcount - 1e15))
        rel_freq = frequency / total
        frequencies = (frequency, rel_freq)
        yield pair, frequencies

if __name__ == '__main__':
    StripesTop50.run()

Writing StripesTop50HW252.py


In [40]:
from StripesTop50HW252 import StripesTop50
mr_job = StripesTop50(args=['ProductPurchaseData.txt']) 
with mr_job.make_runner() as runner: 
    runner.run()
    counter = 1      
    for line in runner.stream_output():
        pair, frequencies =  mr_job.parse_output_line(line)
        print counter, pair[0].upper(), pair[1].upper(), frequencies[0], frequencies[1]
        counter += 1
        if counter > 50:
            break

1 DAI62779 ELE17451 1592 0.0511880646925
2 FRO40251 SNA80324 1412 0.0454004694383
3 DAI75645 FRO40251 1254 0.0403202469374
4 FRO40251 GRO85051 1213 0.0390019613517
5 DAI62779 GRO73461 1139 0.0366226166361
6 DAI75645 SNA80324 1130 0.0363332368734
7 DAI62779 FRO40251 1070 0.0344040384554
8 DAI62779 SNA80324 923 0.0296775023311
9 DAI62779 DAI85309 918 0.0295167357963
10 ELE32164 GRO59710 911 0.0292916626475
11 DAI62779 DAI75645 882 0.0283592167454
12 FRO40251 GRO73461 882 0.0283592167454
13 DAI62779 ELE92920 877 0.0281984502106
14 FRO40251 FRO92469 835 0.026848011318
15 DAI62779 ELE32164 832 0.0267515513971
16 DAI75645 GRO73461 712 0.0228931545609
17 DAI43223 ELE32164 711 0.022861001254
18 DAI62779 GRO30386 709 0.02279669464
19 ELE17451 FRO40251 697 0.0224108549564
20 DAI85309 ELE99737 659 0.0211890292917
21 DAI62779 ELE26917 650 0.020899649529
22 GRO21487 GRO73461 631 0.0202887366966
23 DAI62779 SNA45677 604 0.0194205974084
24 ELE17451 SNA80324 597 0.0191955242597
25 DAI62779 GRO71621 59

Report  the compute times for stripes job versus the Pairs job. Describe the computational setup used (E.g., single computer; dual core; linux, number of mappers, number of reducers)

In [41]:
%%writefile StripesHW251_runnertimer.py
from StripesHW251 import Stripes
mr_job = Stripes(args=['ProductPurchaseData.txt', '--jobconf="mapred.map.tasks=2"', '--jobconf="mapred.reduce.tasks=2"']) 
with mr_job.make_runner() as runner: 
    runner.run()

Writing StripesHW251_runnertimer.py


In [42]:
%time !python StripesHW251_runnertimer.py

CPU times: user 75 ms, sys: 27.6 ms, total: 103 ms
Wall time: 12.5 s


### Machine used was a quad core Mac, running on OS X.  Mapper/Reducer count set to 2.  Number of Mapper, Reducer, Combiner, MapperSort and ReducerSort calls in the results above.  The Stripes Method ran substantially faster than the Pairs method.  Also, the number of combiners and reducers was dramatically reduced.

Instrument your mapper, combiner, and reducer to count how many times each is called using Counters and report these counts. Discuss the differences in these counts between the Pairs and Stripes jobs

In [43]:
!python PairsHW241_runnertimer.py

[{'FunctionCalls': {'Mapper': 31101, 'Reducer': 877095, 'Combiner': 1026707}}]


In [44]:
!python StripesHW251_runnertimer.py

In [45]:
import pandas
from IPython.display import display
pairsdata = {'Mapper Calls': '31101', 'Combiner Calls': '1026707', 'Reducer Calls': '877095'}
pairsDF = pandas.DataFrame({"Metrics": pairsdata.keys(), "Pairs Stats": pairsdata.values()})

stripesdata = {'Mapper Calls': '31101', 'Combiner Calls': '17747', 'Reducer Calls': '12593'}
stripesDF = pandas.DataFrame({"Metrics": stripesdata.keys(), "Stripes Stats": stripesdata.values()})
pair_stripe_comp = pandas.merge(pairsDF, stripesDF, how='inner')

pair_stripe_comp = pandas.merge(pairsDF, stripesDF, how='inner')
reduction = {'Mapper Calls': 31101 - 31101, 'Combiner Calls': 1026707 - 17747, 'Reducer Calls': 877095 - 12593}
reductionDF = pandas.DataFrame({"Metrics": reduction.keys(), "Reduction": reduction.values()})
comparison = pandas.merge(pair_stripe_comp, reductionDF, how='inner')
display(comparison)

,Metrics,Pairs Stats,Stripes Stats,Reduction
0,Mapper Calls,31101,31101,0
1,Reducer Calls,877095,12593,864502
2,Combiner Calls,1026707,17747,1008960


Results of Comparison between Pairs and Stripes:
* Expectedly, the Mapper call was not changed, as the mapper is called for each line in the file.  
* The combiner is called substantially less in the Stripes method because the Pairs method creates significantly more key-value pairs.  
* The Reducer is called substantially less in the Stripes method for the same reason the combiners are reduced (fewer key-value pairs created in Stripes than in Pairs method).